# 🎯 Master Pipeline: End-to-End Data Science Project

<img src='https://miro.medium.com/max/1400/1*KzmIUYPmxgEHhXX7SlbP4w.jpeg' width='600' alt='Pipeline'>

## 🚀 Complete Data Science Workflow

**This is where everything comes together!** We'll build:
- **Complete preprocessing pipeline**
- **Feature engineering automation**
- **Model-ready transformations**
- **Production-grade code**
- **Real business case study**

### 📚 What We'll Build:
1. **Data Pipeline Architecture** - Modular design
2. **Custom Transformers** - Sklearn compatible
3. **Feature Engineering Pipeline** - Automated features
4. **Preprocessing Pipeline** - End-to-end cleaning
5. **Validation Framework** - Quality checks
6. **Model Preparation** - Ready for ML
7. **Pipeline Persistence** - Save and load
8. **Production Deployment** - API ready
9. **Monitoring & Logging** - Track everything
10. **Complete Project** - Customer churn prediction

---

## 🏗️ Let's Build Production Pipelines!

In [2]:
# Import all required  libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import joblib
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("🎯 Master  Pipeline - Ready to Build!")
print("\n💡 Building  production-ready data pipelines!")

c:\Users\RC\miniconda3\envs\ML_Python\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🎯 Master  Pipeline - Ready to Build!

💡 Building  production-ready data pipelines!


---

## 📌 Section 1: Custom Transformers

### 🎯 Building Sklearn-Compatible Components

In [3]:
# 1.1  Custom Transformer Classes
print("🔧 CUSTOM  TRANSFORMERS\n" + "="*40)

class OutlierRemover(BaseEstimator, TransformerMixin):
    """Remove outliers using IQR method"""
    
    def __init__(self, factor=1.5):
        self.factor = factor
        self.bounds_ = {}
    
    def fit(self, X, y=None):
        for column in X.columns:
            if X[column].dtype in ['int64', 'float64']:
                Q1 = X[column].quantile(0.25)
                Q3 = X[column].quantile(0.75)
                IQR = Q3 - Q1
                self.bounds_[column] = (
                    Q1 - self.factor * IQR,
                    Q3 + self.factor * IQR
                )
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for column, (lower, upper) in self.bounds_.items():
            X_copy[column] = X_copy[column].clip(lower, upper)
        logger.info(f"Outliers capped for {len(self.bounds_)} columns")
        return X_copy


class MissingIndicator(BaseEstimator, TransformerMixin):
    """Add binary indicators for missing values"""
    
    def __init__(self, threshold=0.1):
        self.threshold = threshold
        self.columns_ = []
    
    def fit(self, X, y=None):
        missing_pct = X.isnull().sum() / len(X)
        self.columns_ = missing_pct[missing_pct > self.threshold].index.tolist()
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns_:
            X_copy[f'{col}_was_missing'] = X[col].isnull().astype(int)
        logger.info(f"Added {len(self.columns_)} missing indicators")
        return X_copy


class FeatureEngineer(BaseEstimator, TransformerMixin):
    """Create new features from existing ones"""
    
    def __init__(self, create_interactions=True, create_ratios=True):
        self.create_interactions = create_interactions
        self.create_ratios = create_ratios
        self.numeric_columns_ = []
    
    def fit(self, X, y=None):
        self.numeric_columns_ = X.select_dtypes(include=[np.number]).columns.tolist()
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        # Create polynomial features
        for col in self.numeric_columns_:
            if col in X.columns:
                X_copy[f'{col}_squared'] = X[col] ** 2
                X_copy[f'{col}_log'] = np.log1p(np.abs(X[col]))
        
        # Create interaction features
        if self.create_interactions and len(self.numeric_columns_) > 1:
            for i, col1 in enumerate(self.numeric_columns_[:-1]):
                for col2 in self.numeric_columns_[i+1:]:
                    if col1 in X.columns and col2 in X.columns:
                        X_copy[f'{col1}_x_{col2}'] = X[col1] * X[col2]
        
        # Create ratio features
        if self.create_ratios and len(self.numeric_columns_) > 1:
            for i, col1 in enumerate(self.numeric_columns_[:-1]):
                for col2 in self.numeric_columns_[i+1:]:
                    if col1 in X.columns and col2 in X.columns:
                        # Avoid division by zero
                        X_copy[f'{col1}_div_{col2}'] = X[col1] / (X[col2] + 1e-8)
        
        logger.info(f"Created {len(X_copy.columns) - len(X.columns)} new features")
        return X_copy


class CategoryEncoder(BaseEstimator, TransformerMixin):
    """Smart categorical encoding based on cardinality"""
    
    def __init__(self, max_onehot=10):
        self.max_onehot = max_onehot
        self.encoders_ = {}
        self.encoding_type_ = {}
    
    def fit(self, X, y=None):
        for col in X.select_dtypes(include=['object']).columns:
            n_unique = X[col].nunique()
            
            if n_unique <= 2:
                # Binary encoding
                self.encoders_[col] = LabelEncoder()
                self.encoders_[col].fit(X[col].fillna('missing'))
                self.encoding_type_[col] = 'binary'
            elif n_unique <= self.max_onehot:
                # One-hot encoding
                self.encoders_[col] = pd.get_dummies
                self.encoding_type_[col] = 'onehot'
            else:
                # Target encoding (simplified version)
                if y is not None:
                    mean_target = y.mean()
                    self.encoders_[col] = X[col].map(
                        X.groupby(col)[y.name].mean()
                    ).fillna(mean_target)
                    self.encoding_type_[col] = 'target'
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        for col, encoding_type in self.encoding_type_.items():
            if encoding_type == 'binary':
                X_copy[col] = self.encoders_[col].transform(X[col].fillna('missing'))
            elif encoding_type == 'onehot':
                dummies = pd.get_dummies(X[col], prefix=col)
                X_copy = pd.concat([X_copy.drop(col, axis=1), dummies], axis=1)
        
        logger.info(f"Encoded {len(self.encoding_type_)} categorical columns")
        return X_copy


print("✅  Custom transformers created:")
print("  • OutlierRemover")
print("  • MissingIndicator")
print("  • FeatureEngineer")
print("  • CategoryEncoder")

🔧 CUSTOM  TRANSFORMERS
✅  Custom transformers created:
  • OutlierRemover
  • MissingIndicator
  • FeatureEngineer
  • CategoryEncoder


---

## 📌 Section 2: Pipeline Architecture

### 🎯 Building Modular Pipelines

In [4]:
# 2.1 Create Sample Dataset
print("📊 CREATING SAMPLE DATASET\n" + "="*40)

# Generate realistic customer churn dataset
np.random.seed(42)
n_samples = 5000

data = pd.DataFrame({
    'customer_id': range(1, n_samples + 1),
    'age': np.random.normal(45, 15, n_samples).clip(18, 80),
    'tenure_months': np.random.exponential(24, n_samples),
    'monthly_charges': np.random.gamma(2, 40, n_samples),
    'total_charges': np.random.lognormal(7, 1.5, n_samples),
    'num_services': np.random.poisson(3, n_samples),
    'contract_type': np.random.choice(['Month-to-month', 'One year', 'Two year'], n_samples, p=[0.5, 0.3, 0.2]),
    'payment_method': np.random.choice(['Electronic', 'Mailed check', 'Bank transfer', 'Credit card'], n_samples),
    'internet_service': np.random.choice(['DSL', 'Fiber optic', 'No'], n_samples, p=[0.3, 0.5, 0.2]),
    'tech_support': np.random.choice(['Yes', 'No'], n_samples, p=[0.3, 0.7]),
    'online_security': np.random.choice(['Yes', 'No'], n_samples, p=[0.4, 0.6]),
    'device_protection': np.random.choice(['Yes', 'No'], n_samples, p=[0.35, 0.65]),
    'streaming_tv': np.random.choice(['Yes', 'No'], n_samples, p=[0.45, 0.55]),
    'streaming_movies': np.random.choice(['Yes', 'No'], n_samples, p=[0.45, 0.55]),
    'satisfaction_score': np.random.uniform(1, 5, n_samples),
    'support_tickets': np.random.poisson(2, n_samples),
    'late_payments': np.random.poisson(0.5, n_samples)
})

# Add correlations and create target
data['total_charges'] = data['monthly_charges'] * data['tenure_months'] * np.random.uniform(0.8, 1.2, n_samples)

# Create churn target with logical relationships
churn_probability = (
    0.1 +  # Base probability
    0.3 * (data['contract_type'] == 'Month-to-month') +
    0.2 * (data['satisfaction_score'] < 2.5) +
    0.1 * (data['support_tickets'] > 5) +
    0.1 * (data['late_payments'] > 2) +
    0.1 * (data['tenure_months'] < 12) -
    0.2 * (data['contract_type'] == 'Two year') -
    0.1 * (data['online_security'] == 'Yes')
)

data['churn'] = (np.random.random(n_samples) < churn_probability).astype(int)

# Add missing values randomly
missing_cols = ['satisfaction_score', 'support_tickets', 'total_charges']
for col in missing_cols:
    missing_idx = np.random.choice(data.index, size=int(0.1 * len(data)), replace=False)
    data.loc[missing_idx, col] = np.nan

# Add some outliers
outlier_idx = np.random.choice(data.index, size=50, replace=False)
data.loc[outlier_idx, 'monthly_charges'] *= 5
data.loc[outlier_idx[:25], 'total_charges'] *= 10

print(f"Dataset shape: {data.shape}")
print(f"Churn rate: {data['churn'].mean()*100:.1f}%")
print(f"Missing values: {data.isnull().sum().sum()}")
print("\nFirst 5 rows:")
print(data.head())

📊 CREATING SAMPLE DATASET
Dataset shape: (5000, 18)
Churn rate: 31.0%
Missing values: 1500

First 5 rows:
   customer_id        age  tenure_months  monthly_charges  total_charges  \
0            1  52.450712       4.420973        28.998892            NaN   
1            2  42.926035       5.055371        42.243909     250.571324   
2            3  54.715328      14.820140        22.840260     282.019941   
3            4  67.845448       8.094345        49.860711     346.586247   
4            5  41.487699       6.823322        93.380864     562.060393   

   num_services   contract_type payment_method internet_service tech_support  \
0             4  Month-to-month     Electronic      Fiber optic           No   
1             2        Two year  Bank transfer              DSL           No   
2             3  Month-to-month     Electronic               No           No   
3             2        One year    Credit card      Fiber optic           No   
4             1  Month-to-month  Bank

In [6]:
data.head()

,customer_id,age,tenure_months,monthly_charges,total_charges,num_services,contract_type,payment_method,internet_service,tech_support,online_security,device_protection,streaming_tv,streaming_movies,satisfaction_score,support_tickets,late_payments,churn
0,1,52.450712,4.420973,28.998892,NaN,4,Month-to-month,Electronic,Fiber optic,No,Yes,No,Yes,Yes,NaN,2.0,0,0
1,2,42.926035,5.055371,42.243909,250.571324,2,Two year,Bank transfer,DSL,No,Yes,No,No,Yes,4.079777,3.0,0,0
2,3,54.715328,14.820140,22.840260,282.019941,3,Month-to-month,Electronic,No,No,Yes,No,Yes,No,3.974223,2.0,0,0
3,4,67.845448,8.094345,49.860711,346.586247,2,One year,Credit card,Fiber optic,No,No,No,No,No,3.819199,0.0,2,0
4,5,41.487699,6.823322,93.380864,562.060393,1,Month-to-month,Bank transfer,DSL,No,No,Yes,Yes,No,1.042771,4.0,1,1


In [8]:
# 2.2 Build Preprocessing Pipeline
print("🔧 BUILDING PREPROCESSING PIPELINE\n" + "="*40)

# Separate features and target
X = data.drop(['customer_id', 'churn'], axis=1)
y = data['churn']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Define column groups
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumeric features ({len(numeric_features)}): {numeric_features[:5]}...")
print(f"Categorical features ({len(categorical_features)}): {categorical_features[:5]}...")

# Create preprocessing pipelines for numeric and categorical data
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('outliers', OutlierRemover(factor=1.5)),
    ('scaler', RobustScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(drop='first', sparse_output=False))
])

# Combine pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

# Create the full pipeline
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

print("\n📋 Pipeline Structure:")
print("1. Numeric Pipeline:")
print("   → Median Imputation")
print("   → Outlier Capping")
print("   → Robust Scaling")
print("\n2. Categorical Pipeline:")
print("   → Constant Imputation")
print("   → One-Hot Encoding")
print("\n3. Model:")
print("   → Random Forest Classifier")

# Fit the pipeline
print("\n⏳ Training pipeline...")
full_pipeline.fit(X_train, y_train)

# Make predictions
y_pred = full_pipeline.predict(X_test)
y_pred_proba = full_pipeline.predict_proba(X_test)[:, 1]

# Evaluate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n 📊 Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred):.3f}")
print(f"Recall: {recall_score(y_test, y_pred):.3f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.3f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.3f}")

🔧 BUILDING PREPROCESSING PIPELINE
Training set: (4000, 16)
Test set: (1000, 16)

Numeric features (8): ['age', 'tenure_months', 'monthly_charges', 'total_charges', 'num_services']...
Categorical features (8): ['contract_type', 'payment_method', 'internet_service', 'tech_support', 'online_security']...

📋 Pipeline Structure:
1. Numeric Pipeline:
   → Median Imputation
   → Outlier Capping
   → Robust Scaling

2. Categorical Pipeline:
   → Constant Imputation
   → One-Hot Encoding

3. Model:
   → Random Forest Classifier

⏳ Training pipeline...


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

---

## 📌 Section 3: Advanced Pipeline Features

### 🎯 Feature Engineering and Selection

In [5]:
# 3.1 Advanced Feature Engineering Pipeline
print("🚀 ADVANCED FEATURE ENGINEERING\n" + "="*40)

class AdvancedFeatureEngineer(BaseEstimator, TransformerMixin):
    """Advanced feature engineering with domain knowledge"""
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        # Domain-specific features for telecom churn
        if 'monthly_charges' in X.columns and 'tenure_months' in X.columns:
            X_copy['avg_charges_per_month'] = X['total_charges'] / (X['tenure_months'] + 1)
            X_copy['charge_increase_rate'] = (X['monthly_charges'] - X_copy['avg_charges_per_month']) / (X_copy['avg_charges_per_month'] + 1)
        
        if 'num_services' in X.columns:
            X_copy['service_diversity'] = X['num_services'] / X['num_services'].max()
        
        if 'support_tickets' in X.columns and 'tenure_months' in X.columns:
            X_copy['tickets_per_month'] = X['support_tickets'] / (X['tenure_months'] + 1)
        
        if 'satisfaction_score' in X.columns:
            X_copy['is_satisfied'] = (X['satisfaction_score'] >= 3.5).astype(int)
            X_copy['very_dissatisfied'] = (X['satisfaction_score'] < 2).astype(int)
        
        # Contract value features
        if 'contract_type' in X.columns:
            X_copy['is_month_to_month'] = (X['contract_type'] == 'Month-to-month').astype(int)
            X_copy['has_long_contract'] = (X['contract_type'].isin(['One year', 'Two year'])).astype(int)
        
        # Service bundle features
        service_cols = ['internet_service', 'tech_support', 'online_security', 
                       'device_protection', 'streaming_tv', 'streaming_movies']
        available_services = [col for col in service_cols if col in X.columns]
        if available_services:
            X_copy['total_services'] = sum([(X[col] == 'Yes').astype(int) 
                                           for col in available_services 
                                           if col != 'internet_service'])
        
        # Risk indicators
        if 'late_payments' in X.columns:
            X_copy['payment_risk'] = (X['late_payments'] > 0).astype(int)
            X_copy['high_payment_risk'] = (X['late_payments'] > 2).astype(int)
        
        logger.info(f"Created {len(X_copy.columns) - len(X.columns)} domain-specific features")
        return X_copy


class FeatureSelector(BaseEstimator, TransformerMixin):
    """Select best features based on importance"""
    
    def __init__(self, method='mutual_info', k=20):
        self.method = method
        self.k = k
        self.selected_features_ = None
        self.scores_ = None
    
    def fit(self, X, y=None):
        if self.method == 'mutual_info':
            selector = SelectKBest(mutual_info_classif, k=min(self.k, X.shape[1]))
        else:
            selector = SelectKBest(f_classif, k=min(self.k, X.shape[1]))
        
        selector.fit(X, y)
        self.selected_features_ = X.columns[selector.get_support()].tolist()
        self.scores_ = selector.scores_
        return self
    
    def transform(self, X):
        logger.info(f"Selected {len(self.selected_features_)} features from {X.shape[1]}")
        return X[self.selected_features_]


# Create advanced pipeline
advanced_numeric_pipeline = Pipeline([
    ('missing_indicator', MissingIndicator(threshold=0.05)),
    ('imputer', SimpleImputer(strategy='median')),
    ('outliers', OutlierRemover(factor=2.0)),
    ('engineer', AdvancedFeatureEngineer()),
    ('scaler', StandardScaler())
])

# Test advanced pipeline
X_train_advanced = advanced_numeric_pipeline.fit_transform(X_train)
print(f"Original features: {X_train.shape[1]}")
print(f"After engineering: {X_train_advanced.shape[1]}")

# Feature importance analysis
from sklearn.ensemble import RandomForestClassifier

# Train a model to get feature importances
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_advanced, y_train)

# Get feature importances
feature_names = [f"feature_{i}" for i in range(X_train_advanced.shape[1])]
importances = pd.DataFrame({
    'feature': feature_names,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top features
plt.figure(figsize=(10, 6))
top_features = importances.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances')
plt.tight_layout()
plt.show()

print("\n📊 Top 10 Features:")
print(importances.head(10).to_string(index=False))

2025-09-15 09:05:15,878 - INFO - Added 3 missing indicators


🚀 ADVANCED FEATURE ENGINEERING


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'Two year'

---

## 📌 Section 4: Cross-Validation and Hyperparameter Tuning

### 🎯 Optimizing Pipeline Performance

In [ ]:
# 4.1 Cross-Validation and Grid Search
print("🔍 PIPELINE OPTIMIZATION\n" + "="*40)

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Create a simpler pipeline for faster optimization
optimization_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Define parameter grid
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__num__scaler': [StandardScaler(), RobustScaler()],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [5, 10, None],
    'classifier__min_samples_split': [2, 5, 10]
}

print("🔍 Starting Grid Search...")
print(f"Total combinations: {np.prod([len(v) for v in param_grid.values()])}")

# Perform grid search with cross-validation
grid_search = GridSearchCV(
    optimization_pipeline,
    param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

# Fit grid search
grid_search.fit(X_train, y_train)

print("\n✅ Grid Search Complete!")
print(f"Best Score: {grid_search.best_score_:.3f}")
print("\nBest Parameters:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# Evaluate best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]

print("\n📊 Best Model Performance on Test Set:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.3f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_best):.3f}")

# Cross-validation scores
from sklearn.model_selection import cross_validate

cv_scores = cross_validate(
    best_model,
    X_train,
    y_train,
    cv=5,
    scoring=['accuracy', 'precision', 'recall', 'roc_auc'],
    return_train_score=True
)

# Display CV results
cv_results = pd.DataFrame(cv_scores)
print("\n📊 Cross-Validation Results (5-fold):")
print(cv_results[['test_accuracy', 'test_precision', 'test_recall', 'test_roc_auc']].describe().round(3))

---

## 📌 Section 5: Pipeline Persistence

### 🎯 Saving and Loading Pipelines

In [ ]:
# 5.1 Save and Load Pipeline
print("💾 PIPELINE PERSISTENCE\n" + "="*40)

import joblib
import pickle
from datetime import datetime
import json

# Create model metadata
model_metadata = {
    'model_name': 'churn_prediction_pipeline',
    'version': '1.0.0',
    'created_date': datetime.now().isoformat(),
    'model_type': type(best_model).__name__,
    'features': list(X_train.columns),
    'performance': {
        'roc_auc': roc_auc_score(y_test, y_pred_proba_best),
        'accuracy': accuracy_score(y_test, y_pred_best),
        'precision': precision_score(y_test, y_pred_best),
        'recall': recall_score(y_test, y_pred_best)
    },
    'training_data_shape': X_train.shape,
    'parameters': grid_search.best_params_
}

# Save pipeline using joblib
pipeline_filename = 'churn_prediction_pipeline.pkl'
metadata_filename = 'pipeline_metadata.json'

# Save the pipeline
joblib.dump(best_model, pipeline_filename)
print(f"✅ Pipeline saved to {pipeline_filename}")

# Save metadata
with open(metadata_filename, 'w') as f:
    json.dump(model_metadata, f, indent=2)
print(f"✅ Metadata saved to {metadata_filename}")

# Display metadata
print("\n📋 Model Metadata:")
for key, value in model_metadata.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for k, v in value.items():
            print(f"  {k}: {v}")
    elif isinstance(value, list) and len(value) > 5:
        print(f"{key}: [{value[0]}, {value[1]}, ... ] ({len(value)} items)")
    else:
        print(f"{key}: {value}")

# Load and test pipeline
print("\n🔄 Loading pipeline...")
loaded_pipeline = joblib.load(pipeline_filename)

# Test loaded pipeline
sample_data = X_test.iloc[:5]
predictions = loaded_pipeline.predict(sample_data)
probabilities = loaded_pipeline.predict_proba(sample_data)[:, 1]

print("\n✅ Pipeline loaded successfully!")
print("\nSample predictions:")
results_df = pd.DataFrame({
    'Actual': y_test.iloc[:5].values,
    'Predicted': predictions,
    'Probability': probabilities.round(3)
})
print(results_df)

---

## 📌 Section 6: Production Deployment

### 🎯 API-Ready Pipeline

In [ ]:
# 6.1 Create Production-Ready Prediction Function
print("🚀 PRODUCTION DEPLOYMENT\n" + "="*40)

class ChurnPredictor:
    """Production-ready churn prediction class"""
    
    def __init__(self, model_path, metadata_path):
        self.model = joblib.load(model_path)
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
        self.features = self.metadata['features']
        logger.info(f"Model loaded: {self.metadata['model_name']} v{self.metadata['version']}")
    
    def validate_input(self, data):
        """Validate input data"""
        if not isinstance(data, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame")
        
        missing_features = set(self.features) - set(data.columns)
        if missing_features:
            raise ValueError(f"Missing features: {missing_features}")
        
        return data[self.features]
    
    def predict(self, data):
        """Make predictions"""
        try:
            # Validate input
            data = self.validate_input(data)
            
            # Make predictions
            predictions = self.model.predict(data)
            probabilities = self.model.predict_proba(data)[:, 1]
            
            # Create response
            response = {
                'predictions': predictions.tolist(),
                'probabilities': probabilities.tolist(),
                'model_version': self.metadata['version'],
                'timestamp': datetime.now().isoformat()
            }
            
            logger.info(f"Prediction completed for {len(data)} samples")
            return response
            
        except Exception as e:
            logger.error(f"Prediction error: {str(e)}")
            raise
    
    def predict_single(self, customer_data):
        """Predict for a single customer"""
        df = pd.DataFrame([customer_data])
        result = self.predict(df)
        
        return {
            'churn_prediction': 'Yes' if result['predictions'][0] == 1 else 'No',
            'churn_probability': result['probabilities'][0],
            'risk_level': self._get_risk_level(result['probabilities'][0]),
            'timestamp': result['timestamp']
        }
    
    def _get_risk_level(self, probability):
        """Categorize risk level"""
        if probability < 0.3:
            return 'Low'
        elif probability < 0.7:
            return 'Medium'
        else:
            return 'High'
    
    def get_model_info(self):
        """Get model information"""
        return self.metadata


# Initialize predictor
predictor = ChurnPredictor(pipeline_filename, metadata_filename)

# Test single prediction
test_customer = X_test.iloc[0].to_dict()
result = predictor.predict_single(test_customer)

print("📊 Single Customer Prediction:")
for key, value in result.items():
    print(f"  {key}: {value}")

# Batch prediction
batch_result = predictor.predict(X_test.iloc[:10])
print(f"\n📊 Batch Prediction ({len(batch_result['predictions'])} customers):")
print(f"  Churn rate: {np.mean(batch_result['predictions'])*100:.1f}%")
print(f"  Average probability: {np.mean(batch_result['probabilities']):.3f}")

---

## 📌 Section 7: Monitoring and Validation

### 🎯 Pipeline Monitoring

In [ ]:
# 7.1 Monitoring and Validation
print("📊 PIPELINE MONITORING\n" + "="*40)

class PipelineMonitor:
    """Monitor pipeline performance and data drift"""
    
    def __init__(self, reference_data):
        self.reference_data = reference_data
        self.reference_stats = self._calculate_stats(reference_data)
        self.monitoring_history = []
    
    def _calculate_stats(self, data):
        """Calculate statistics for monitoring"""
        stats = {}
        
        # Numeric features
        numeric_cols = data.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            stats[col] = {
                'mean': data[col].mean(),
                'std': data[col].std(),
                'min': data[col].min(),
                'max': data[col].max(),
                'missing_pct': data[col].isnull().mean()
            }
        
        # Categorical features
        categorical_cols = data.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            stats[col] = {
                'unique_values': data[col].nunique(),
                'mode': data[col].mode()[0] if not data[col].mode().empty else None,
                'missing_pct': data[col].isnull().mean()
            }
        
        return stats
    
    def detect_drift(self, new_data, threshold=0.1):
        """Detect data drift"""
        new_stats = self._calculate_stats(new_data)
        drift_report = {}
        
        for feature in self.reference_stats:
            if feature in new_stats:
                ref = self.reference_stats[feature]
                new = new_stats[feature]
                
                if 'mean' in ref:  # Numeric feature
                    mean_change = abs(new['mean'] - ref['mean']) / (ref['mean'] + 1e-8)
                    std_change = abs(new['std'] - ref['std']) / (ref['std'] + 1e-8)
                    
                    if mean_change > threshold or std_change > threshold:
                        drift_report[feature] = {
                            'mean_change': mean_change,
                            'std_change': std_change,
                            'status': 'DRIFT DETECTED'
                        }
        
        return drift_report
    
    def validate_predictions(self, predictions, expected_range=(0, 1)):
        """Validate prediction outputs"""
        validation_report = {
            'total_predictions': len(predictions),
            'null_predictions': np.isnan(predictions).sum(),
            'out_of_range': ((predictions < expected_range[0]) | 
                           (predictions > expected_range[1])).sum(),
            'mean_prediction': np.mean(predictions),
            'std_prediction': np.std(predictions)
        }
        
        # Validation status
        if validation_report['null_predictions'] > 0:
            validation_report['status'] = 'FAILED - Null predictions found'
        elif validation_report['out_of_range'] > 0:
            validation_report['status'] = 'WARNING - Out of range predictions'
        else:
            validation_report['status'] = 'PASSED'
        
        return validation_report
    
    def log_monitoring_event(self, event_type, details):
        """Log monitoring events"""
        event = {
            'timestamp': datetime.now().isoformat(),
            'event_type': event_type,
            'details': details
        }
        self.monitoring_history.append(event)
        logger.info(f"Monitoring event: {event_type}")
        return event

# Initialize monitor
monitor = PipelineMonitor(X_train)

# Test drift detection
print("🔍 Drift Detection Test:")
drift_report = monitor.detect_drift(X_test)
if drift_report:
    print("⚠️ Drift detected in features:")
    for feature, details in drift_report.items():
        print(f"  {feature}: {details['status']}")
else:
    print("✅ No significant drift detected")

# Validate predictions
test_predictions = predictor.predict(X_test)['probabilities']
validation_report = monitor.validate_predictions(test_predictions)

print("\n📊 Prediction Validation:")
for key, value in validation_report.items():
    print(f"  {key}: {value}")

# Performance tracking
def track_model_performance(y_true, y_pred, y_proba):
    """Track model performance metrics over time"""
    
    performance_metrics = {
        'timestamp': datetime.now().isoformat(),
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_proba),
        'confusion_matrix': confusion_matrix(y_true, y_pred).tolist()
    }
    
    return performance_metrics

# Track current performance
current_performance = track_model_performance(y_test, y_pred_best, y_pred_proba_best)

print("\n📈 Current Model Performance:")
for metric, value in current_performance.items():
    if metric != 'confusion_matrix' and metric != 'timestamp':
        print(f"  {metric}: {value:.3f}")

---

## 📌 Section 8: Complete Production Pipeline

### 🎯 Putting It All Together

In [ ]:
# 8.1 Complete Production Pipeline Class
print("🏗️ COMPLETE PRODUCTION PIPELINE\n" + "="*40)

class ProductionPipeline:
    """Complete production-ready ML pipeline"""
    
    def __init__(self, config=None):
        self.config = config or self._default_config()
        self.pipeline = None
        self.monitor = None
        self.metadata = {}
        self.performance_history = []
        
    def _default_config(self):
        return {
            'model_name': 'production_pipeline',
            'version': '1.0.0',
            'test_size': 0.2,
            'random_state': 42,
            'cv_folds': 5,
            'scoring_metric': 'roc_auc',
            'drift_threshold': 0.1,
            'feature_engineering': True,
            'hyperparameter_tuning': True
        }
    
    def build_pipeline(self, X, y):
        """Build complete preprocessing and model pipeline"""
        
        logger.info("Building production pipeline...")
        
        # Identify feature types
        numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X.select_dtypes(include=['object']).columns.tolist()
        
        # Create preprocessing pipelines
        numeric_transformer = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', RobustScaler())
        ])
        
        categorical_transformer = Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ])
        
        # Combine preprocessing
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])
        
        # Create full pipeline
        self.pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(random_state=self.config['random_state']))
        ])
        
        # Store metadata
        self.metadata['features'] = {
            'numeric': numeric_features,
            'categorical': categorical_features,
            'total': len(numeric_features) + len(categorical_features)
        }
        
        logger.info(f"Pipeline built with {self.metadata['features']['total']} features")
        return self
    
    def train(self, X, y):
        """Train the pipeline"""
        
        logger.info("Starting training...")
        
        # Split data
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, 
            test_size=self.config['test_size'],
            random_state=self.config['random_state'],
            stratify=y
        )
        
        # Build pipeline if not exists
        if self.pipeline is None:
            self.build_pipeline(X_train, y_train)
        
        # Train pipeline
        self.pipeline.fit(X_train, y_train)
        
        # Validate
        y_pred = self.pipeline.predict(X_val)
        y_proba = self.pipeline.predict_proba(X_val)[:, 1]
        
        # Calculate metrics
        metrics = {
            'accuracy': accuracy_score(y_val, y_pred),
            'precision': precision_score(y_val, y_pred),
            'recall': recall_score(y_val, y_pred),
            'f1_score': f1_score(y_val, y_pred),
            'roc_auc': roc_auc_score(y_val, y_proba)
        }
        
        # Store results
        self.metadata['training'] = {
            'train_size': len(X_train),
            'val_size': len(X_val),
            'validation_metrics': metrics,
            'trained_at': datetime.now().isoformat()
        }
        
        # Initialize monitor
        self.monitor = PipelineMonitor(X_train)
        
        logger.info(f"Training complete. ROC-AUC: {metrics['roc_auc']:.3f}")
        return metrics
    
    def predict(self, X):
        """Make predictions with monitoring"""
        
        if self.pipeline is None:
            raise ValueError("Pipeline not trained. Call train() first.")
        
        # Check for drift
        if self.monitor:
            drift_report = self.monitor.detect_drift(X, self.config['drift_threshold'])
            if drift_report:
                logger.warning(f"Data drift detected in {len(drift_report)} features")
        
        # Make predictions
        predictions = self.pipeline.predict(X)
        probabilities = self.pipeline.predict_proba(X)[:, 1]
        
        return {
            'predictions': predictions,
            'probabilities': probabilities,
            'timestamp': datetime.now().isoformat()
        }
    
    def save(self, filepath):
        """Save pipeline and metadata"""
        
        # Save pipeline
        joblib.dump(self.pipeline, f"{filepath}_pipeline.pkl")
        
        # Save metadata
        with open(f"{filepath}_metadata.json", 'w') as f:
            json.dump(self.metadata, f, indent=2)
        
        logger.info(f"Pipeline saved to {filepath}")
    
    def load(self, filepath):
        """Load pipeline and metadata"""
        
        self.pipeline = joblib.load(f"{filepath}_pipeline.pkl")
        
        with open(f"{filepath}_metadata.json", 'r') as f:
            self.metadata = json.load(f)
        
        logger.info(f"Pipeline loaded from {filepath}")
        return self


# Test complete pipeline
print("\n🧪 Testing Complete Pipeline:")

# Initialize
prod_pipeline = ProductionPipeline()

# Train
metrics = prod_pipeline.train(X, y)

print("\n📊 Training Results:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.3f}")

# Make predictions
test_results = prod_pipeline.predict(X_test)

print(f"\n✅ Predictions made for {len(test_results['predictions'])} samples")
print(f"  Average probability: {np.mean(test_results['probabilities']):.3f}")

# Save pipeline
prod_pipeline.save('production')
print("\n💾 Pipeline saved successfully!")

---

## 📌 Section 9: API Simulation

### 🎯 REST API Endpoint Simulation

In [ ]:
# 9.1 API Endpoint Simulation
print("🌐 API ENDPOINT SIMULATION\n" + "="*40)

def api_predict(request_data):
    """Simulate API endpoint for predictions"""
    
    try:
        # Validate request
        if not request_data or 'data' not in request_data:
            return {
                'status': 'error',
                'message': 'Invalid request. Missing data field.',
                'code': 400
            }
        
        # Convert data to DataFrame
        df = pd.DataFrame(request_data['data'])
        
        # Make predictions
        results = prod_pipeline.predict(df)
        
        # Format response
        response = {
            'status': 'success',
            'code': 200,
            'data': {
                'predictions': results['predictions'].tolist(),
                'probabilities': results['probabilities'].tolist(),
                'model_version': prod_pipeline.config['version'],
                'timestamp': results['timestamp']
            },
            'metadata': {
                'samples_processed': len(results['predictions']),
                'model_name': prod_pipeline.config['model_name']
            }
        }
        
        return response
        
    except Exception as e:
        return {
            'status': 'error',
            'message': str(e),
            'code': 500
        }


# Test API endpoint
print("📡 Testing API Endpoint:\n")

# Prepare test request
test_request = {
    'data': X_test.iloc[:3].to_dict('records')
}

print("Request:")
print(f"  Sending {len(test_request['data'])} samples")

# Make API call
response = api_predict(test_request)

print("\nResponse:")
print(f"  Status: {response['status']}")
print(f"  Code: {response['code']}")
if response['status'] == 'success':
    print(f"  Predictions: {response['data']['predictions']}")
    print(f"  Probabilities: {[round(p, 3) for p in response['data']['probabilities']]}")
    print(f"  Model Version: {response['data']['model_version']}")

# Test error handling
print("\n🔍 Testing Error Handling:")
bad_request = {'invalid': 'data'}
error_response = api_predict(bad_request)
print(f"  Error Status: {error_response['status']}")
print(f"  Error Message: {error_response['message']}")

---

## 📌 Section 10: Pipeline Documentation

### 🎯 Complete Documentation and Best Practices

In [ ]:
# 10.1 Generate Pipeline Documentation
print("📚 PIPELINE DOCUMENTATION\n" + "="*40)

def generate_documentation(pipeline, metadata):
    """Generate comprehensive pipeline documentation"""
    
    doc = f"""
# Production Pipeline Documentation
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Pipeline Overview
- **Name**: {metadata.get('model_name', 'N/A')}
- **Version**: {metadata.get('version', 'N/A')}
- **Purpose**: Customer Churn Prediction
- **Type**: Binary Classification

## Features
### Numeric Features ({len(metadata['features']['numeric'])})
{', '.join(metadata['features']['numeric'][:5])}...

### Categorical Features ({len(metadata['features']['categorical'])})
{', '.join(metadata['features']['categorical'][:5])}...

## Model Performance
### Validation Metrics
"""
    
    if 'training' in metadata and 'validation_metrics' in metadata['training']:
        for metric, value in metadata['training']['validation_metrics'].items():
            doc += f"- **{metric.upper()}**: {value:.3f}\n"
    
    doc += f"""

## Pipeline Components
1. **Data Validation**
   - Missing value checks
   - Data type validation
   - Range validation

2. **Preprocessing**
   - Numeric: Median imputation, Robust scaling
   - Categorical: Mode imputation, One-hot encoding
   - Outlier handling: IQR-based capping

3. **Feature Engineering**
   - Polynomial features
   - Interaction terms
   - Domain-specific features

4. **Model**
   - Algorithm: Random Forest Classifier
   - Hyperparameters: Optimized via GridSearch

## Usage Examples

### Python
```python
# Load pipeline
pipeline = joblib.load('pipeline.pkl')

# Make predictions
predictions = pipeline.predict(data)
probabilities = pipeline.predict_proba(data)[:, 1]
```

### API
```bash
curl -X POST http://api.example.com/predict \\
  -H "Content-Type: application/json" \\
  -d '{"data": [{...customer_features...}]}'
```

## Monitoring
- Data drift detection enabled
- Performance tracking enabled
- Logging configured

## Maintenance
- **Retraining Schedule**: Monthly
- **Performance Threshold**: ROC-AUC < 0.75
- **Data Requirements**: Minimum 1000 samples

## Contact
- **Team**: Data Science Team
- **Email**: datascience@example.com
"""
    
    return doc

# Generate documentation
documentation = generate_documentation(prod_pipeline.pipeline, prod_pipeline.metadata)

# Save documentation
with open('pipeline_documentation.md', 'w') as f:
    f.write(documentation)

print("📄 Documentation generated and saved!")
print("\nDocumentation Preview:")
print("=" * 50)
print(documentation[:800] + "...")

# Best practices checklist
print("\n✅ BEST PRACTICES CHECKLIST:")
print("=" * 50)

checklist = [
    "✓ Data validation implemented",
    "✓ Missing value handling",
    "✓ Outlier detection and treatment",
    "✓ Feature scaling applied",
    "✓ Categorical encoding handled",
    "✓ Cross-validation performed",
    "✓ Hyperparameter tuning done",
    "✓ Model persistence implemented",
    "✓ Monitoring framework in place",
    "✓ API endpoint created",
    "✓ Documentation generated",
    "✓ Error handling implemented",
    "✓ Logging configured",
    "✓ Version control ready"
]

for item in checklist:
    print(item)

---

## 🎯 Summary & Next Steps

### 🏆 What You've Built:

✅ **Custom Transformers**
- OutlierRemover
- MissingIndicator
- FeatureEngineer
- CategoryEncoder

✅ **Production Pipeline**
- Modular architecture
- Automated preprocessing
- Feature engineering
- Model training

✅ **Advanced Features**
- Cross-validation
- Hyperparameter tuning
- Performance monitoring
- Data drift detection

✅ **Deployment Ready**
- Pipeline persistence
- API endpoints
- Error handling
- Documentation

✅ **Best Practices**
- Logging
- Validation
- Testing
- Monitoring

### 🚀 Next Steps:

1. **Deploy to Production** - Use Flask/FastAPI
2. **Add More Models** - Ensemble methods
3. **Implement A/B Testing** - Compare models
4. **Setup CI/CD** - Automated deployment
5. **Add Real-time Monitoring** - Dashboard

### 💡 Key Takeaways:

- **Pipelines ensure reproducibility**
- **Automation reduces errors**
- **Monitoring prevents degradation**
- **Documentation enables collaboration**

### 📚 Resources:

- Scikit-learn Pipeline Documentation
- MLOps Best Practices
- Production ML Systems
- Feature Engineering Techniques

---

## 🎉 Congratulations!

You've built a complete production-ready ML pipeline!

This is how real data science projects are deployed.

**Keep building, keep deploying, keep improving!** 🚀

In [ ]:
# 🎊 Module Complete!
print("🎊" * 20)
print("\n    🏆 MASTER PIPELINE COMPLETE! 🏆")
print("\n    You've mastered:")
print("    ✅ Custom Transformers")
print("    ✅ Pipeline Architecture")
print("    ✅ Feature Engineering")
print("    ✅ Model Deployment")
print("    ✅ Production Best Practices")
print("\n    Ready for: Machine Learning Algorithms!")
print("\n" + "🎊" * 20)